# Delta Method

## Introduction

The Delta method, often used in AB Testing, is a general method for approximating the standard error of the Average Treatment Effect (ATE) estimator when the estimand (the metric) is defined in a way that violates traditional Central Limit theorem rules, which is what is typically used for performing inference (deriving the standard error of the estimate based on Asymptotic normality).

For example, when your metric is a percent change (e.g., $\frac{\bar Y_T - \bar Y_c}{\bar Y_c}$), where $\bar Y_T$ is the Mean in Treatment and $\bar Y_C$ is the Mean in Control, the Central Limit does not apply because the CLT, traditionally, only applies for sums (linear functions) of iid random variables, not ratios.

However, there are several popular metrics where the metric-of-interest is a ratio, or other types that do not typically obey CLT. Experimentation Platforms, including with **StatSig**, rely on the *Delta Method* for these scenarios, which extends the Central Limit Theorem to allow inference with Ratio and other tricky metrics in a way that is scalable and easy.

This notebook is intended to be *self-contained*, so we will review not only the Delta Method but relevant concepts part of understanding the theory behind the Delta Method.

## Delta Method

*Taken from [Wikipedia](https://en.wikipedia.org/wiki/Delta_method), adapted to be much more "beginner-friendly".*

The Delta method says that, if there is a sequence of random variables $X_n$ satisfying

$$\sqrt{n}\left[X_n-\theta\right]\xrightarrow{D}\mathcal{N}(0,\sigma^2),$$

where $\theta,\sigma^2$ are finite valued constants and $\xrightarrow{D}$ denotes convergence in distribution, then

$$\sqrt{n}\left[g(X_n)-g(\theta)\right]\xrightarrow{D}\mathcal{N}(0,\sigma^2\cdot\left[g'(\theta)\right]^2),$$

for any function $g$ satisfying the property that its first derivative, evaluated at $\theta$, $g'(\theta)$ exists and is non-zero valued.

Let's discuss in a bit more detail. The Delta Method says that if the first condition is satisfied, then the second condition satifies too. The first condition says that sequence $X_n$ converges to normality. A "sequence" $X_n$ is like a "list of random variables", and can be something like a "sample mean of $n$ coin tosses", so a random variable that is represented by $n$ , a process that evolves as $n$ increases. In practice, we have a clear example of this. For example, the Delta method is used for Percent Change metrics, where the metric is defined as $\frac{\bar Y_T - \bar Y_C}{\bar Y_C}$. Each of those components (numerator and denominator) is individually normal because they obey CLT. As you will see later, in the case of multiple random variables, we use a "multivariate Delta method", but it is quite analogous to the univarate variant. In this case, the $g$ can be thought of as the nonlinear transformation of the 2 variables that make it a ratio.

The intuition of the delta method is that any such $g$ function, in a "small enough" range of the function, can be approximated via a first order Taylor series (which is basically a linear function). If the random variable is roughly normal then a linear transformation of it is also normal.

In other words, by using a First-Order Taylor series, which turns $g(X_n)$ into a linear function (locally), we can then invoke the CLT again, with the 2nd formula being the result of First-Order Taylor Series. This is because one property of Normal random variables is that a Linear Transformation of a Normal random variable is also a Normal random variable.

The text also states that small range can be achieved when approximating the function around the mean, when the variance is "small enough". When $g$ is applied to a random variable such as the mean, the delta method would tend to work better as the sample size increases, since it would help reduce the variance, and thus the Taylor approximation would be applied to a smaller range of the function $g$ at the point of interest.

This means that for the random variable $X_n$, if we were to use a typical mean as we do in AB tests, then because it tends to be centered around the true value $\theta$, and that it becomes more precise as sample size grows, the fact that the Taylor series is linear "locally" around the true value is justified in its use.





## Proof in the Univariate Case

*Taken from [Wikipedia](https://en.wikipedia.org/wiki/Delta_method), adapted to be much more "beginner-friendly".*

Demonstration of this result is straightforward under the assumption that $g(x)$ is differentiable near the neighborhood of $\theta$ and $g'(x)$ is continuous at $\theta$ with $g'(\theta)\neq 0$.

First, we use the First-Order Approximation of a Taylor Series:

$$g(X_n)=g(\theta)+g'(\tilde\theta)(X_n-\theta)$$

where $\tilde\theta$ lies between $X_n$ and $\theta$. This comes from the "Mean Value Theorem", which is a form of Taylor's theorem and is valid under the assumption that $g$ is differentiable in a neighborhood of $\theta$.

Now, because [convergence in distribution to a constant implies convergence in probability to that constant](https://math.stackexchange.com/questions/1716298/proof-for-convergence-in-distribution-implying-convergence-in-probability-for-co), $X_n\xrightarrow{P}\theta$. Also, $|\tilde\theta-\theta|<|X_n-\theta|$, it must be that $\tilde\theta\xrightarrow{P}\theta$ and since $g'(\theta)$ is continuous, the *continuous mapping theorem* yields

$$g'(\tilde\theta)\xrightarrow{P}g'(\theta)$$, where $\xrightarrow{P}$ denotes convergence in probability. Formally, the continuous mapping theorem states that if:
- $X_n\xrightarrow{P}x$
- $f$ is continuous at $x$

Then, $f(X_n)\xrightarrow{P}f(x)$.

Rearranging the terms and multiplying by $\sqrt{n}$ gives

$$\sqrt{n}\left[g(X_n)-g(\theta)\right]=g'(\tilde\theta)\sqrt{n}\left[X_n-\theta\right]$$.

Before moving on, Slutsky's theorem says that if:
- $A_n\xrightarrow{D}A$
- $B_n\xrightarrow{P}b$

Then, $A_n\cdot B_n\xrightarrow{D}A\cdot b$.

Now, since $\sqrt{n}\left[X_n-\theta\right]\xrightarrow{D}\mathcal{N}(0,\sigma^2)$ by assumption, it follows from appeal to Slutsky's theorem:
- $\sqrt{n}\left[X_n-\theta\right]\xrightarrow{D}\mathcal{N}(0,\sigma^2)$
- $g'(\tilde\theta)\xrightarrow{P}g'(\theta)$

Thus, $g'(\tilde\theta)\sqrt{n}\left[X_n-\theta\right]\xrightarrow{D}g'(\theta)\cdot\mathcal{N}(0,\sigma^2)$.

Finally, using the fact that $K\cdot\mathcal{N}(0,\sigma^2) = \mathcal{N}(0,K^2\sigma^2)$ and substitution,

$$\sqrt{n}\left[g(X_n)-g(\theta)\right]\xrightarrow{D}\mathcal{N}(0,\sigma^2\cdot\left[g'(\theta)\right]^2),$$.

$\text{End of Proof.}$

# Delta Method in the Context of AB Testing

*Adapted from [Deng and Knoblich (2018)](https://arxiv.org/pdf/1803.06336)*

In this section, we'll explore the Delta Method in the context of Online Experimentation at Scale.

Deng and Knoblich discuss how in AB Testing, we typically rely on metrics that are traditional "Average Treatment Effect" metrics where the estimand neatly obeys the Central Limit Theorem, allowing for quick and intuitive inference via a t-test.

For example, the Central Limit Theorem says that if we have $X_1,...,X_n$ iid observations with finite mean $\mu$ and variance $\sigma^2 > 0$ and we denote $\bar X$ as the sample average, then as sample size $n\rightarrow\infty$, in distribution

$$\sqrt{n}(\bar X-\mu)/\sigma\rightarrow N(0,1)$$.

A common application of the CLT is to construct the $100(1-\alpha)\%$ confidence interval of $\mu$ as $\bar X\pm z_{\alpha/2}\sigma/\sqrt{n}$, where $z_{\alpha/2}$ is the $(\alpha/2)$th quantile for $N(0,1)$.

Note that in an Average Treatment Effect, it is the difference of 2 means, and the difference of 2 normal distributions is also a normal distribution. Thus, inference on the ATE is straightforward.

However, not all metrics of interest are the difference of 2 means, where each mean is sums or averages of iid observations. In such cases, we employ the Delta method, which extends the normal approximations from the CLT more broadly. Abandoning previous mathematical notation to follow this papers, for any random variable $T_n$ (the subscript indicates its dependency on $n$, e.g., sample average) and constant $\theta$ such that $\sqrt{n}(T_n-\theta)\rightarrow N(0,1)$ in distribution as $n\rightarrow\infty$, the Delta method allows us to extend its asymptotic normality to any continuous transformation $\phi(T_n)$.

To be more specific, by using the fact that $T_n-\theta=O(1/\sqrt{n})$ and the first order Taylor expansion

$$\phi(T_n)-\phi(\theta)=\phi'(\theta)(T_n-\theta)+O\{(T_n-\theta)^2\}$$,

we have in distribution

$$\sqrt{n}\{\phi(T_n)-\theta(\theta)\}\rightarrow N\{0,\phi'(\theta)^2\}$$

To elaborate a bit further (as we did not discuss this in the previous theory section), the first order Taylor expansion gives us a remainder term $O(\cdot)$ that vanishes faster than the Central Limit Theorem of $O(1/\sqrt{n})$, at a rate of $O(1/n)$. This means that in Asymptotic inference, it is inconsequential, this error will vanish faster than the convergence rate of the Central Limit Theorem and thus it does not matter.



## Percent Changes / Ratio Metrics

Let $X_1,...,X_n$ be iid observations from the control group with mean $\mu_x$ and variance $\sigma_x^2$, and $Y_1,...,Y_n$ iid observations from the treatment group with mean $\mu_y$ and variance $\sigma_y^2$, and $\sigma_{xy}$ the covariance ($0$ in AB tests) between $X_i$'s and $Y_j$s. Let $\bar X=\sum_{i=1}^{n}X_i/n$ and $\bar Y=\sum_{i=1}^{n}Y_i/n$ be two measurements of the same metric from the treatment and control groups, and their difference $\hat\Delta=\bar Y-\bar X$ is an unbiased estimate of the ATE $\Delta=\mu_y-\mu_x$. Because both $\bar X, \bar Y$ are approximately normally distributed, their difference $\hat\Delta$ also follows an approximate normal distribution with mean $\Delta$ and variance

$$\text{Var}(\hat\Delta)=(\sigma_y^2+\sigma_x^2-2\sigma_{xy})/n$$

Consequently, the well-known $100(1-\alpha)\%$ confidence interval of $\delta$ is $\hat\delta\pm z_{\alpha}\times\hat{Var}(\hat\Delta)$.

In practice however, *absolute* differences are hard to interpret because they are not scale-invariant. Thus, there is some interest in focusing on the *relative* difference or percent change $\Delta\%=(\mu_y-\mu_x)/\mu_x$, estimated by $\hat\Delta\%=(\bar Y-\bar X)/\bar X$. The key problem is constructing a confidence interval for $\hat\Delta\%$, as we cannot rely on the typical Central Limit Theorem result here, as CLT applies to sums and averages, not ratios of random variables.

Traditionally, Fieller (1954) derives a method for computing the standard error here (not shown here), but it is complex and cumbersome to derive. This is where the Delta method comes in.

## Delta method Derivation for Ratio Metrics

The Delta method provides a more intuitive alternative solution.

Let's derive the standard error below.

Let:
- $T_n=(\bar Y, \bar X)$
- $\theta=(\mu_y,\mu_x)$
- $\phi(x,y)=y/x$

The multivariate first order taylor expansion is:

$$f(x)\approx f(a)+\nabla f(a)^T(x-a)$$

(This looks about the same as the univariate version)

Thus, when applied to our case, we have:

$$\phi(\bar Y,\bar X)\approx\phi(\mu_x,\mu_y)+\frac{\partial\phi}{\partial\bar Y}\big|_{(\mu_y,\mu_x)}(\bar Y-\mu_y)+\frac{\partial\phi}{\partial\bar X}\big|_{(\mu_y,\mu_x)}(\bar X-\mu_x)$$

Now we need to derive these partial derivatives of $\phi=\bar Y/\bar X$ w.r.t $\bar Y$ and $\bar X$:
- w.r.t $\bar Y\rightarrow\frac{\partial\phi}{\partial\bar Y}=\frac{1}{\bar X}$
- w.r.t $\bar X\rightarrow\frac{\partial\phi}{\partial\bar X}=-\frac{\bar Y}{\bar X^2}$

Now, evaluate them at $\theta$ and plug it back into the expression to get:

$$\frac{\bar Y}{\bar X}\approx\frac{\mu_y}{\mu_x}+\frac{1}{\mu_x}(\bar Y-\mu_y)-\frac{\mu_y}{\mu_x^2}(\bar X-\mu_x)$$

We can apply the delta method directly by applying $\nabla g^T\Sigma\nabla g$, alternatively, we can compute the variance directly using algebraic rules for playing with variance terms:

$$\text{Var}\left(\frac{\bar Y}{\bar X}\right)\approx\text{Var}\left(\frac{\mu_y}{\mu_x}+\frac{1}{\mu_x}(\bar Y-\mu_y)-\frac{\mu_y}{\mu_x^2}(\bar X-\mu_x)\right)$$

After expanding it, we can eliminate a lot of terms because $\theta=(\mu_y,\mu_x)$ has 0 variance since they are fixed. Then, we have:

$$\text{Var}\left(\frac{\bar Y}{\bar X}\right)\approx\text{Var}\left(\frac{1}{\mu_x}\bar Y-\frac{\mu_y}{\mu_x^2}\bar X\right)$$

Recall this rule for the variance of a linear combination:

$$\text{Var}(aU-bV)=a^2\text{Var}(U)+b^2\text{Var}(V)-2ab\text{Cov}(U,V)$$

Apply this to our data and we get:

$$\text{Var}\left(\frac{\bar Y}{\bar X}\right)\approx\frac{1}{\mu_x^2}\text{Var}\left(\bar Y\right)-\left(\frac{\mu_y}{\mu_x^2}\right)^2\text{Var}\left(\bar X\right)-2\frac{1}{\mu_x}\frac{\mu_y}{\mu_x^2}\text{Cov}(\bar Y,\bar X)$$

Now, we substitute:
- $\text{Var}(\bar Y)=\sigma_y^2/n$
- $\text{Cov}(\bar Y,\bar X)=\sigma_{xy}/n$

To get:

$$\text{Var}\left(\frac{\bar Y}{\bar X}\right)\approx\frac{1}{\mu_x^2}\frac{\sigma_y^2}{n}+\left(\frac{\mu_y}{\mu_x^2}\right)^2\frac{\sigma_x^2}{n}-2\frac{1}{\mu_x}\frac{\mu_y}{\mu_x^2}\frac{\sigma_{xy}}{n}$$

Next, we factor out common terms and apply the plug-in principle, replacing the population parameters $\mu,\sigma$ with their empirical analogs (sample means and variances). This is justified by Slutsky’s theorem, which ensures that substituting consistent estimators for constants within an expression that converges in distribution preserves that convergence. Since the sample estimates converge in probability to their true values, their inclusion does not affect the limiting distribution.

$$\text{Var}\left(\frac{\bar{Y}}{\bar{X}}\right)
\approx \frac{1}{n\bar{X}^2} \left[
s_y^2
- 2 \cdot \frac{\bar{Y}}{\bar{X}} \cdot s_{xy}
+ \left( \frac{\bar{Y}}{\bar{X}} \right)^2 \cdot s_x^2
\right]$$

That's all! Now, before we show the full confidence interval result, you may be wondering, "isn't our estimand supposed to be $\frac{\bar Y-\bar X}{\bar X}$, not $\frac{\bar Y}{\bar X}$? This is because their variances are equivalent. To see this, notice how:

$$\text{Var}\left(\frac{\bar Y-\bar X}{\bar X}\right)=\text{Var}\left(\frac{\bar Y}{\bar X}-\frac{\bar X}{\bar X}\right)=\text{Var}\left(\frac{\bar Y}{\bar X}-1\right)=\text{Var}\left(\frac{\bar Y}{\bar X}\right)$$

Thus, our Confidence Interval is:

$$\frac{\bar Y}{\bar X}-1\pm z_{\alpha/2}\cdot\frac{1}{\sqrt{n}\bar X}\sqrt{s_y^2
- 2 \cdot \frac{\bar{Y}}{\bar{X}} \cdot s_{xy}
+ \left( \frac{\bar{Y}}{\bar{X}} \right)^2 \cdot s_x^2
}$$





## Cluster Randomization

Two key concepts in a typical AB test are the *randomization unit* and the *analysis unit*.

The Randomization unit refers to the granularity level where the randomization is performed, and the analysis unit is the aggregation level of metric computation.

Often, the two are the same, which makes analysis straightforward. However, we can see scenarios where they diverse. Note that the analysis unit must be the same level or *more granular* than randomization, otherwise, the analysis unit may contain observations from both control and treatment. For example, if we randomize by User but our analysis unit is city, that wouldn't make much sense, would it? On the other hand, if we randomize by city, but analyze by user (e.g., average signup rate), we can do it, but not without a standard t-test as "iid" would be violated. Once again, the standard CLT assumptions are invalidated.

A few more cases is that there is need to reduce bias from network effects, and another is that the same experiment, while randomized by user, might have metrics with different analysis units to meet different business needs. For example, having both user-level and page-level metrics.

We consider 2 average metrics of the treatment and control groups, assumed to be independent via randomization. Without loss of generality, we only focus on the Treatment group with $K$ clusters.

For $i=1,...,K$, the $i$th cluster contains $N_i$ analysis unit level observations $Y_{ij} (j=1,..,N_i)$.

Then, the corresponding average metric is $\bar Y=\frac{\sum_{i,j}Y_{ij}}{\sum_i N_i}$, which can be interpreted as the total average of all rows of data. Remember, the # of rows correspond to the # of analysis units.

We assume that within each cluster $i$, the observations $Y_{ij}$'s are iid with mean $\mu_i$ and variance $\sigma_i^2$, and across clusters $(\mu_i, \sigma_i,N_i)$ are also iid.

To make this easier, think of clusters as Users, and analysis units as page views. We have randomized by users, but we are interested in page-view metrics like "Page-Load-Time" or "Page Crash Rate".

Traditional solutions for calculating variances for $\text{Var}(\bar Y)$ exist, dating back to Allan Donner (1987). While not shown here, the formual is incredibly restrictive due to the unrealistic assumptions. For example, it requires equal cluster sizes amongst all users (e.g., # of pages each user loaded is the same) and that the average page load times are the same across all users.

Another common approach is the Mixed Effects Model (i.e., multi-level / hierarchical regression, Gelman and Hill, 2006). Under this setting, we can infer the treatment effect as the "fixed" effect for the treatment indicator term. Stan (2019) offers a Markov Chain Monte Carlo implementation, which requires significant computational effort for big data. Moreover, the estimate ATE is for the randomization unit but not the analysis unit level. This means that we can craft estimands like "*difference in average page load time per user between treatment and control*" but not "*average treatment effect per pageview across the entire sample*".









## Delta Method for Clustered Randomization

Recall the following notation:

For $i=1,...,K$, the $i$th cluster contains $N_i$ analysis unit level observations $Y_{ij} (j=1,..,N_i)$.

Then, the corresponding average metric is $$\bar Y=\frac{\sum_{i,j}Y_{ij}}{\sum_i N_i}$$.

Equivalently, we can rewrite $$\bar Y=\frac{\sum_i(\sum_jY_{ij})}{\sum_iN_i}$$

Let $S_i=\sum_iY_{ij}$, and divide both the numerator and denominator by $K$:

$$\bar Y=\frac{\sum_iS_i/K}{\sum_iN_i/K}=\bar S/\bar N$$

Now, $\bar Y$ is a ratio of two averages of iid randomization unit level quantities!.

Then, we can use the same derivation from before:

$$\text{Var}(\bar{Y}) \approx \frac{1}{K \mu_N^2} \left( \sigma_S^2 - 2 \frac{\mu_S}{\mu_N} \sigma_{SN} + \frac{\mu_S^2}{\mu_N^2} \sigma_N^2 \right)$$

There's a few interesting implications here. For example, $\sigma^2_N=\text{Var}(N_i)$ implies that there is some desire to make the cluster *sizes* more homogeneous - as if they are a lot of variation, it leads to more variation in the estimate of $\bar Y$.


# Simulation

First, we'll copy the Data Generation from Deng (2018), which creates a dataset of $K$ rows, one for each cluster (the randomization unit), indexed with $i$.

First, we decide if each row is going to be a Small, Medium, or High cluster via a Multinomial Distribution (generalizes the Binomial distribution)

$$(M_1,M_2,M_3)'\sim\text{Multi-Nomial}\left(n=K;p=\left(\frac{1}{3},\frac{1}{2},\frac{1}{6}\right)\right)$$

Then, once the size for the $i$th row/cluster is decided, we then give them their size and values:
- $\text{Small}:N_i\sim\text{Poisson}(2),\mu_i\sim\text{N}(\mu=0.3,\sigma=0.05)$
- $\text{Medium}:N_i\sim\text{Poisson}(5),\mu_i\sim\text{N}(\mu=0.5,\sigma=0.1)$
- $\text{High}:N_i\sim\text{Poisson}(30),\mu_i\sim\text{N}(\mu=0.8,\sigma=0.05)$

Now, for each cluster $i$, we also need output values:

$$Y_{ij}\sim\text{Bernoulli}(p=\mu_i)$$ for all $j=1,...,N_i$.

To get the "truth" value, we compute the weighted average of $\mu_i$ weighted
by the cluster sizes and the mixture of small, medium and large
clusters.

Let's think about this more intuitively. A naive approach would be to take the average of the 3 $\mu$'s and then multiply by total # of clusters, $K$, to get the numerator $\sum_iS_i$. This wouldn't work, because this assumes that the distribution of the different kinds of $\mu$'s are even.

We then instead weigh it by how often that cluster appears, but also multiply how big the clusters are:

$$(1/3)*2*0.3+(1/2)*5*0.5+(1/6)*30*0.8$$

Now we need the denominator, $\sum_iN_i$. This is the same as the numerator, but is a count, so you can think of it as all the $\mu$'s being $1$. Thus,

$$(1/3)*2+(1/2)*5+(1/6)*30$$

Put it together: this equals $\approx 0.667$

Let's now code this out.

In [1]:
import numpy as np
import scipy.stats as stats
from statsmodels.regression.mixed_linear_model import MixedLM
import pandas as pd

def delta_variance(N_i, mu_i, sigma_i):
  """
  Computes Variance using Delta Method. i stands for each randomization unit. Should be K rows.
  """
  # num. of clusters, i.e., num. rows
  K = len(N_i)

  # empirical mean and var. of N_i
  mu_N = np.mean(N_i)
  sigma_N2 = np.var(N_i, ddof=1)

  # empirical mean and var. of S_i
  mu_S = np.mean(N_i * mu_i)
  sigma_S2 = np.var(N_i * mu_i, ddof=1)

  # covariance of s_i and n_i
  sigma_SN = np.cov(N_i * mu_i, N_i, ddof=1)[0, 1]

  # Delta Variance
  var = (1 / (K * mu_N**2)) * (sigma_S2 - 2* (mu_S / mu_N) * sigma_SN + (mu_S**2 / mu_N**2) * sigma_N2)
  return var

def simulate_run():
  # Step 1: Assign Cluster Categories
  K = 1000
  M = np.random.multinomial(K, [1/3, 1/2, 1/6]) # Small, Medium High
  cluster_ids = []
  sizes = []
  mu_vals = []
  sigma_vals = []

  # Step 2: Generate N_i, mu_i, sigma_i for each category
  categories = [('small', 2, 0.3, 0.05),
                ('medium', 5, 0.5, 0.1),
                ('high', 30, 0.8, 0.05)]

  for m, (label, lam, mu_mean, sigma) in zip(M, categories): # this loop is a 3 x 5 table used as a "source of truth" to create the dataset
    N_i = np.random.poisson(lam, m)
    mu_i = np.random.normal(mu_mean, sigma, m)
    mu_i = np.clip(mu_i, 0, 1) # To ensure they are valid Bernoulli
    cluster_ids.extend(range(len(cluster_ids), len(cluster_ids) + m))
    sizes.extend(N_i)
    mu_vals.extend(mu_i)
    sigma_vals.extend([sigma]*m) # sigmas are constant across each "cluster"-type. So there are 3 different sigmas.

  cluster_ids = np.array(cluster_ids)
  sizes = np.array(sizes)
  mu_vals = np.array(mu_vals)
  sigma_vals = np.array(sigma_vals)

  # Step 3: Generate observations Y_{ij} ~ Bernoulli(mu_i)
  Y_all = [] # after the loop, becomes of size np.sum(sizes)
  cluster_label = [] # used to make sure we don't lose track what cluster_id each of the values in Y_all belongs to.
  for i in range(len(cluster_ids)):
    Y_all.extend(np.random.binomial(1, mu_vals[i], sizes[i]))
    cluster_label.extend([i] * sizes[i])

  Y_all = np.array(Y_all)
  cluster_label = np.array(cluster_label)
  total_obs = len(Y_all)

  # Step 4: Estimators
  naive_mean = np.mean(Y_all)
  naive_var_individual = np.var(Y_all, ddof=1) # Variance of individual observations
  naive_var_mean = naive_var_individual / total_obs # Naive variance of the mean

  # Mixed model
  data = pd.DataFrame({
      'Y': Y_all,
      'cluster': cluster_label
  })
  try:
      model = MixedLM.from_formula("Y ~ 1", groups="cluster", data=data)
      result = model.fit(reml=False)
      mixed_mean = result.params["Intercept"]
      mixed_var = result.bse["Intercept"] ** 2
  except:
      mixed_mean = np.nan
      mixed_var = np.nan

  # Delta method
  delta_var = delta_variance(sizes, mu_vals, np.array(sigma_vals))
  delta_mean = naive_mean

  return {
      'naive_mean': naive_mean,
      'naive_var_mean': naive_var_mean, # Return the naive variance of the mean
      'delta_mean': delta_mean,
      'delta_var': delta_var,
      'mixed_mean': mixed_mean,
      'mixed_var': mixed_var,
      'total_obs': total_obs # Return total observations for verification
  }

In [2]:
def run_simulation(n_sim=1000, true_mean=0.667):
    results = []
    for i in range(n_sim):
        results.append(simulate_run())
        if i % 100 == 0:
            print(f'Simulation {i}/{n_sim} complete')

    df = pd.DataFrame(results)

    def coverage(mean, var):
        lower = mean - 1.96 * np.sqrt(var)
        upper = mean + 1.96 * np.sqrt(var)
        return (lower <= true_mean) & (upper >= true_mean)

    summary = {
        'method': [],
        'mean_est': [],
        'empirical_sd': [],
        'avg_est_var': [],
        'coverage_95': []
    }

    for method in ['naive', 'delta', 'mixed']:
        means = df[f'{method}_mean']
        # Use the calculated variance of the mean for naive method
        vars_ = df[f'{method}_var_mean'] if method == 'naive' else df[f'{method}_var']

        coverage_vals = coverage(means, vars_)
        avg_var = vars_.mean()

        summary['method'].append(method)
        summary['mean_est'].append(means.mean())
        summary['empirical_sd'].append(means.std())
        summary['avg_est_var'].append(np.mean(avg_var))
        summary['coverage_95'].append(np.mean(coverage_vals))

    return pd.DataFrame(summary)

In [3]:
run_simulation()

Simulation 0/1000 complete
Simulation 100/1000 complete
Simulation 200/1000 complete
Simulation 300/1000 complete
Simulation 400/1000 complete
Simulation 500/1000 complete
Simulation 600/1000 complete
Simulation 700/1000 complete
Simulation 800/1000 complete
Simulation 900/1000 complete


,method,mean_est,empirical_sd,avg_est_var,coverage_95
0,naive,0.666449,0.009091,0.000027,0.742
1,delta,0.666449,0.009091,0.000060,0.905
2,mixed,0.545916,0.010064,0.000098,0.000
